In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import codecs
import glob

ocr_texts = []
gs_texts = []

ocr_lengths = []
gs_lengths = []


ocr_files = glob.glob('/home/jvdzwaan/data/ocr/BAO*.ocr.txt')
ocr_files.sort()
gs_files = glob.glob('/home/jvdzwaan/data/ocr/BAO*.gs.txt')
gs_files.sort()

for ocr_file, gs_file in zip(ocr_files, gs_files):
    #print ocr_file
    #print gs_file
    with codecs.open(ocr_file, encoding='utf-8') as f:
        ocr_text = f.read()
    ocr_texts.append(ocr_text.lower())
    ocr_lengths.append(len(ocr_text))
    
    with codecs.open(gs_file, encoding='utf-8') as f:
        gs_text = f.read()
    gs_texts.append(gs_text.lower())
    gs_lengths.append(len(gs_text))
    #print len(ocr_text), len(gs_text)
    
print 'Number of newspaper articles:', len(ocr_texts)

In [ ]:
raw_text = []
for ocr, gs in zip(ocr_texts, gs_texts):
    raw_text.append(ocr)
    raw_text.append(gs)
raw_text.append('\n')

raw_text = ' '.join(raw_text)

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))    

n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

In [ ]:
print chars
print char_to_int

In [ ]:
import numpy as np

seq_length = 25
dataX = []
dataY = []
for ocr, gs in zip(ocr_texts, gs_texts):
    text_length = max(len(ocr), len(gs))
    #print text_length
    #print len(ocr),
    if len(ocr) < text_length:
        ocr = ocr + '\n' * (text_length-len(ocr))
    #print len(ocr)
    #print len(gs),
    if len(gs) < text_length:
        gs = gs + '\n' * (text_length-len(gs))
    #print len(gs)
    for i in range(0, text_length-seq_length +1, 1):
        seq_in = ocr[i:i+seq_length]
        seq_out = gs[i:i+seq_length]
        dataX.append(seq_in)
        dataY.append(seq_out)

X = np.zeros((len(dataX), seq_length, n_vocab), dtype=np.bool)
Y = np.zeros((len(dataY), seq_length, n_vocab), dtype=np.bool)

for i, sentence in enumerate(dataX):
    #print sentence
    for j, c in enumerate(sentence):
        X[i, j, char_to_int[c]] = 1
    #print X[i]
    #print X[i].shape

for i, sentence in enumerate(dataY):
    #print sentence
    for j, c in enumerate(sentence):
        Y[i, j, char_to_int[c]] = 1
    #print Y[i]
    #print Y[i].shape

n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
LAYERS = 1

model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE
# note: in a situation where your input sequences have a variable length,
# use input_shape=(None, nb_feature).
model.add(LSTM(seq_length, input_shape=(seq_length, len(chars))))
# For the decoder's input, we repeat the encoded input for each time step
model.add(RepeatVector(seq_length))
# The decoder RNN could be multiple layers stacked or a single layer
for _ in range(LAYERS):
    model.add(LSTM(seq_length, return_sequences=True))

# For each of step of the output sequence, decide which character should be chosen
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
filepath="/home/jvdzwaan/data/tmp/dncvu-ad-mtm/{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, Y, epochs=20, batch_size=100, callbacks=callbacks_list)

In [ ]:
# load the network weights
filename = "/home/jvdzwaan/data/tmp/dncvu-ad-mtm/00-0.0497.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')